In [3]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

BASE = "https://www.u-cursos.cl/ingenieria/2/novedades_institucion/?p="
N_PAGES = 10

# Scrap Raw Data

In [10]:
def get_page_posts(page_num):
    r = requests.get(f"{BASE}{page_num}")
    soup = BeautifulSoup(r.content, "html.parser")
    posts = soup.find_all("div", class_="post objeto")

    return [
        {
            "titulo": p.find("h1").get_text().split("\n")[0].strip(),
            "url": p.find("a", href=True)["href"],
            "mensaje": " ".join([
                "\n" if e.name == "br" else e if isinstance(e, str) else e.get_text()
                for e in p.find("div", class_="msg").find("span", class_="ta")
            ]).replace("\n ", "\n"),
            "fecha": p.find("span", class_="only-excel").get_text()
        }
        for p in posts
    ]

In [11]:
page_iter = range(N_PAGES)

posts_all = [
    get_page_posts(i)
    for i in tqdm(page_iter, total=len(page_iter))
]

  0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
df = pd.DataFrame([
    p
    for pp in posts_all
    for p in pp
]).dropna(subset=["mensaje"]).copy()

print(df.shape)
df.head()

(100, 4)


,titulo,url,mensaje,fecha
0,Talleres Psicoeducativos - Dirección de Salud ...,detalle?id=72101,En nombre de la Dirección de Salud Estudianti...,2023-10-19 14:58:19
1,Programa Estudiantes Promotores de Salud Menta...,detalle?id=72097,En nombre de el área comunitaria de la Direcci...,2023-10-19 14:53:19
2,HOY-19 hrs. 🚀 Torneo de Innovación UChile: ¿So...,detalle?id=72077,¡HOLA 👋🏽! \nDentro del Torneo de Innovación In...,2023-10-19 10:33:19
3,ÚLTIMO PLAZO postulación becas EPEC Estudiante...,detalle?id=72053,\nEstimado y estimada Estudiante de Programas...,2023-10-19 08:55:19
4,Grabación Charla Doble Título 2024-2026,detalle?id=72049,"Estimados/as Estudiantes, \nSi no pudiste cone...",2023-10-18 17:10:18


In [13]:
df.to_csv("../../data/raw/scraper_test.csv", index=False)